# Review: Querying an API endpoint

### Mapbox Geocoding API

Services like Google Maps and Mapbox have various APIs that let you access its services through code instead of through GUI apps. This one from Mapbox lets you look up the latitude-longitude coordinates of street addresses.

It works similarly to the earthquakes example, but with query parameters added to the URL endpoint!

**API documentation:**  
https://www.mapbox.com/api-documentation/#geocoding

**API endpoint:**  
https://api.mapbox.com/geocoding/v5/mapbox.places

**API endpoint with query parameters:**  
https://api.mapbox.com/geocoding/v5/mapbox.places/Wurster+Hall.json?access_token=pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w

You can get your own access key by signing up for a Mapbox account, if you'd like. Here is a link for that (but don't do it now): https://www.mapbox.com/signin/?route-to=%22/account/access-tokens%22

In [8]:
import json      # library for working with JSON-formatted text strings
import requests  # library for accessing content from web URLs

import pprint    # library for cleanly printing Python data structures
pp = pprint.PrettyPrinter()

In [15]:
# we have to encode the search query so that it can be passed as a URL, 
# with spaces and other special characters removed

endpoint = 'https://api.mapbox.com/geocoding/v5/mapbox.places/'

address = 'Wurster Hall'

params = {'limit': 1,
          'access_token': 'pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w'}

url = requests.Request('GET', endpoint+address+'.json', params=params).prepare().url
print(url)

https://api.mapbox.com/geocoding/v5/mapbox.places/Wurster%20Hall.json?limit=1&access_token=pk.eyJ1IjoiY3AyNTVkZW1vIiwiYSI6ImRPcTlnTUEifQ.3C0d0Nk_rcwV-8JF29PU-w


In [16]:
# download and parse the results

response = requests.get(url)
results = response.text
data = json.loads(results)

print(data)

{'type': 'FeatureCollection', 'query': ['wurster', 'hall'], 'features': [{'id': 'poi.1359351', 'type': 'Feature', 'place_type': ['poi'], 'relevance': 1, 'properties': {'tel': '(510) 642-0831', 'address': '230 Wurster Hall #1820', 'category': 'college, university', 'landmark': True, 'maki': 'college'}, 'text': 'Wurster Hall / College of Environmental Design', 'place_name': 'Wurster Hall / College of Environmental Design, 230 Wurster Hall #1820, Berkeley, California 94720, United States', 'center': [-122.25488, 37.87082], 'geometry': {'type': 'Point', 'coordinates': [-122.25488, 37.87082]}, 'context': [{'id': 'postcode.1038777018848930', 'text': '94720'}, {'id': 'place.4062647275990170', 'wikidata': 'Q484678', 'text': 'Berkeley'}, {'id': 'region.3591', 'short_code': 'US-CA', 'wikidata': 'Q99', 'text': 'California'}, {'id': 'country.3145', 'short_code': 'us', 'wikidata': 'Q30', 'text': 'United States'}]}], 'attribution': 'NOTICE: © 2018 Mapbox and its suppliers. All rights reserved. Use o

In [17]:
# print it more nicely

pp.pprint(data)

{'attribution': 'NOTICE: © 2018 Mapbox and its suppliers. All rights reserved. '
                'Use of this data is subject to the Mapbox Terms of Service '
                '(https://www.mapbox.com/about/maps/). This response and the '
                'information it contains may not be retained.',
 'features': [{'center': [-122.25488, 37.87082],
               'context': [{'id': 'postcode.1038777018848930', 'text': '94720'},
                           {'id': 'place.4062647275990170',
                            'text': 'Berkeley',
                            'wikidata': 'Q484678'},
                           {'id': 'region.3591',
                            'short_code': 'US-CA',
                            'text': 'California',
                            'wikidata': 'Q99'},
                           {'id': 'country.3145',
                            'short_code': 'us',
                            'text': 'United States',
                            'wikidata': 'Q30'}],
          

In [18]:
# pull out the lat-lon coordinates

for r in data['features']:
    coords = r['geometry']['coordinates']
    print(coords)

[-122.25488, 37.87082]


# Using Mapbox Python SDK for Geocoding

So far the discussion of APIs has been based on just accessing the API endpoints, which could be done with anything that can access a properly specified url.  Even just your browser.

Now we will look at APIs from a different perspective, one that is more Pythonic.  We will first need to install the Mapbox SDK.

In [12]:
#!pip install mapbox

You will need to manage access tokens for Mapbox APIs (and for many others).  Read this for more information:

https://github.com/mapbox/mapbox-sdk-py/blob/master/docs/access_tokens.md

In [2]:
from mapbox import Geocoder

In [3]:
geocoder = Geocoder()

In [4]:
response = geocoder.forward('Chester, NJ')

In [10]:
results = response.text

In [11]:
data = json.loads(results)
pp.pprint(data)

{'message': 'Not Authorized - No Token'}
